# Recommender System Case Study

In [1]:
import numpy as np
import pandas as pd

In [8]:
ratings = pd.read_csv('D:\\IITM course till April 23 end Python 3\\New folder\\Module-9-1\\ratings.csv', usecols=['userId','movieId','rating','timestamp'])
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [7]:
movies = pd.read_csv('D:\\IITM course till April 23 end Python 3\\New folder\\Module-9-1\\movies.csv', usecols=['movieId','title','genres'])
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
n_users = ratings.userId.unique().shape[0]

n_movies = ratings.movieId.unique().shape[0]

print(f'Number of users = {n_users} and Number of movies = {n_movies}')

Number of users = 7120 and Number of movies = 14026


In [10]:
Ratings = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
Ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
conda install -c conda-forge scikit-surprise

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.1




In [14]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [17]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
ratings.head()

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8430  0.8464  0.8446  0.8447  0.0014  
MAE (testset)     0.6460  0.6490  0.6463  0.6471  0.0013  
Fit time          9.44    9.27    10.35   9.69    0.47    
Test time         3.08    3.01    3.58    3.22    0.26    


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [18]:
ratings_1 = ratings[(ratings['userId'] == 5) & (ratings['rating'] == 5)]
ratings_1 = ratings_1.set_index('movieId')
ratings_1 = ratings_1.join(movies)['title']
ratings_1.head(10)

movieId
11                    Dracula: Dead and Loving It (1995)
62     Don't Be a Menace to South Central While Drink...
141                                         Gospa (1995)
150                                Addiction, The (1995)
260                             Ladybird Ladybird (1994)
318    Strawberry and Chocolate (Fresa y chocolate) (...
364                                      Maverick (1994)
368                                 Reality Bites (1994)
377                      When a Man Loves a Woman (1994)
380                                   Bad Company (1995)
Name: title, dtype: object

In [19]:
user_5 = movies.copy()
user_5 = user_5.reset_index()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
svd.fit(trainset)
user_5['Estimate_Score'] = user_5['movieId'].apply(lambda x: svd.predict(1, x).est)
user_5 = user_5.drop(['movieId','genres','index'], axis = 1)
user_5 = user_5.sort_values('Estimate_Score', ascending=False)
print(user_5.head(10))

                                                   title  Estimate_Score
7041   Lord of the Rings: The Return of the King, The...        4.681651
5853       Lord of the Rings: The Two Towers, The (2002)        4.665956
4897   Lord of the Rings: The Fellowship of the Ring,...        4.608288
2851                        Palm Beach Story, The (1942)        4.548710
2849                                Lady Eve, The (1941)        4.546209
3003                                      Matewan (1987)        4.528114
10286                                    Serenity (2005)        4.525312
2486                                  Matrix, The (1999)        4.507425
3188                         Boondock Saints, The (2000)        4.504740
12204                Elite Squad (Tropa de Elite) (2007)        4.499726
